In [ ]:
from pathlib import Path

import pandas as pd
import numpy as np

import importlib

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

from tqdm import tqdm
import time

In [2]:
import RSS_bagging as bag

# 2. Inputs

In [3]:
csv_path = Path(r"H:\Meu Drive\Artigo_Mestrado\03_RSS\RSS_candidateSamples_filtered_20250612_reflectances.csv")

n_iterations = 1000
n_bag = 1350
n_classes = 4

In [4]:
output_dir = Path(f"H:/Meu Drive/Artigo_Mestrado/03_RSS/Outputs_{time.strftime("%Y%m%d")}")
output_dir.mkdir(parents=True, exist_ok=True)

In [5]:
df = pd.read_csv(csv_path, sep=",")

## 3. Functions

In [6]:
# Compute mIoU
def compute_miou(conf_matrix):
    intersection = np.diag(conf_matrix)
    union = conf_matrix.sum(axis=1) + conf_matrix.sum(axis=0) - intersection
    iou = intersection / union
    return np.nanmean(iou)

## 4. Classifications

In [7]:
importlib.reload(bag)

bagging = bag.RSS_Bagging(df = df,
    n_bag = n_bag,
    n_classes = n_classes)

df_bag = bagging.create_bag()
df_bag = df_bag[['blue', 'green', 'red', 'nir', 'ndvi', 'modal_class', 'prop','weight']]

name_bag = Path(csv_path.parent, csv_path.stem + f"_bagged_{n_bag}_{n_classes}.csv")
df_bag.to_csv(name_bag, index=False)

In [ ]:
props = np.arange(0.5, 1.01, 0.05)
setups = np.arange(1, 7)
classifier = ['SVM', 'KNN']

In [9]:
for clf in classifier:
    print(f"Running classifier: {clf}")
    for setup in tqdm(setups, desc="Setup"):
        print(f"\tRunning setup: {setup}")
        dict_results = {
        "setup": [],
        "prop": [],
        "accuracy": [],
        "std_accuracy": [],
        "confusion_matrix": [],
        "std_confusion_matrix": [],
        "mIou": [],
        "std_mIou": []}

        for prop in tqdm(props, desc=f"Prop {setup}"):
            prop = np.round(prop, 2)
            dict_eval = {
                "accuracy": [],
                "std_accuracy": [],
                "mIou": [],
                "std_mIou": [],
                "confusion_matrix": [],
                "std_confusion_matrix": []
            }

            for i in tqdm(range(n_iterations),desc="Iteration"):

                train, val = bagging.create_folds(df_bag=df_bag, prop = prop, setup = setup, random_state=i)

                X_train = train[['blue', 'green', 'red', 'nir', "ndvi"]]
                y_train = train['modal_class']
                X_weight = train['weight']

                X_val = val[['blue', 'green', 'red', 'nir', "ndvi"]]
                y_val = val['modal_class']
                X_val_weight = val['weight']

                # Train random forest model
                if classifier == 'RFR':
                    model = RandomForestClassifier(n_estimators=100, random_state=i, criterion='gini', max_depth=10, min_samples_split=2, min_samples_leaf=1, n_jobs=8)
                    model.fit(X_train, y_train, sample_weight=X_weight)
                elif classifier == 'SVM':
                    model = SVC(kernel='rbf', class_weight='balanced', random_state=i)
                    model.fit(X_train, y_train, sample_weight=X_weight)
                elif classifier == 'KNN':
                    model = KNeighborsClassifier(n_neighbors=5, n_jobs=8)
                    model.fit(X_train, y_train, sample_weight=X_weight)

                # Predict on validation set
                y_pred = model.predict(X_val)
                # Evaluate the model
                accuracy = accuracy_score(y_val, y_pred)
                conf_matrix = confusion_matrix(y_val, y_pred)
                mIou = compute_miou(conf_matrix)

                dict_eval['accuracy'].append(accuracy)
                dict_eval['std_accuracy'].append(np.std(y_pred == y_val))
                dict_eval['confusion_matrix'].append(conf_matrix)
                dict_eval['std_confusion_matrix'].append(np.std(conf_matrix, axis=0))
                dict_eval['mIou'].append(mIou)
                dict_eval['std_mIou'].append(np.std(mIou))

            output_prop = Path(output_dir, f"RSS_{clf}_{setup}_{np.round(prop, 2)}_all.txt")
            # Write results to a text file
            with open(output_prop, 'w') as f:
                f.write(f"Setup: {setup}\n")
                f.write(f"Proportion: {prop}\n")
                f.write(f"Accuracy: {dict_eval['accuracy']}\n")
                f.write(f"Confusion Matrix:\n{dict_eval['confusion_matrix']}\n")
                f.write(f"mIoU: {dict_eval['mIou']}\n")

            dict_results['setup'].append(setup)
            dict_results['prop'].append(prop)
            dict_results['accuracy'].append(np.mean(dict_eval['accuracy']))
            dict_results['std_accuracy'].append(np.std(dict_eval['accuracy']))
            dict_results['confusion_matrix'].append(np.mean(dict_eval['confusion_matrix'], axis=0))
            dict_results['std_confusion_matrix'].append(np.std(dict_eval['confusion_matrix'], axis=0))
            dict_results['mIou'].append(np.mean(dict_eval['mIou']))
            dict_results['std_mIou'].append(np.std(dict_eval['mIou']))

            output_restuls = Path(output_dir, f"RSS_{clf}_{setup}_{np.round(prop, 2)}_stats.txt")

            # Write results to a text file
            with open(output_restuls, 'w') as f:
                f.write(f"Setup: {setup}\n")
                f.write(f"Proportion: {prop}\n")
                f.write(f"Mean Accuracy: {dict_results['accuracy'][-1]}\n")
                f.write(f"Std Accuracy: {dict_results['std_accuracy'][-1]}\n")
                f.write(f"Mean Confusion Matrix:\n{dict_results['confusion_matrix'][-1]}\n")
                f.write(f"Std Confusion Matrix:\n{dict_results['std_confusion_matrix'][-1]}\n")
                f.write(f"Mean mIoU: {dict_results['mIou'][-1]}\n")
                f.write(f"Std mIoU: {dict_results['std_mIou'][-1]}\n")


Setup:   0%|          | 0/6 [00:00<?, ?it/s]



























































































































































































































































































































































































































































































































































































































































































































































































































































































































































































